In [112]:
%matplotlib inline

import numpy as np
import pandas as pd


# Plotting imports & settings
import seaborn as sns
from collections import OrderedDict
from matplotlib import pyplot as plt
from pylab import rcParams
plt.style.use('ggplot')
plt_defaults = OrderedDict((('width', 12), ('height', 6)))
rcParams['figure.figsize'] = plt_defaults.values()
rcParams['legend.frameon'] = True
rcParams['legend.framealpha'] = 0.8
rcParams['figure.titlesize'] = 'large'
rcParams['font.size'] = 12
rcParams['legend.fontsize'] = 12

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.externals import joblib


In [136]:
X = pd.read_csv("./data/train_set.csv")
# for i, row in enumerate(X.columns):
# #     if X.dtypes[row] == "object":
# #         del X[row]
#     if row != 'label' and row != 'delivery_method' and row != 'body_length' and row != 'sale_duration':
#         del X[row]
colls = ['delivery_method', 'body_length', 'sale_duration']
colls = ['delivery_method', 'body_length', 'sale_duration', 'delivery_method',
'event_published'    ,
'gts'                ,
'has_header'         ,
'org_facebook'       ,
'org_twitter'        ,
'sale_duration'      ,
'venue_latitude'     ,
'venue_longitude',
              'approx_payout_date', #   int64
              'body_length'       , #    int64
              'channels'          , #    int64
              'event_created'     , #    int64
              'event_end'         , #    int64
              'event_start'       , #    int64
              'fb_published'      , #    int64
              'has_analytics'     , #    int64
              'has_logo'          , #    int64
              'name_length'       , #    int64
              'num_order'         , #    int64
              'num_payouts'       , #    int64
              'object_id'         , #    int64
              'sale_duration2'    , #    int64
              'show_map'          , #    int64
              'user_age'          , #    int64
              'user_created'       #    int64
              # 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
    # 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed',
]
for coll in colls:
    X[coll] = X[coll].fillna(0.0)

X['delivery_method'] = X.delivery_method.fillna(0.0)
X['body_length'] = X.body_length.fillna(0.0)
X['sale_duration'] = X.sale_duration.fillna(0.0)
y = pd.Series(X["label"])
X = X.drop('label',axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=400, random_state=7)

rf = RandomForestClassifier()

rf.fit(X_train[colls], y_train)

joblib.dump(rf, 'filename2.pkl')


['filename2.pkl']

In [126]:
query_df = pd.DataFrame()
query_df.loc[0,'delivery_method'] = 1.0
query_df.loc[0,'body_length'] = 2113
query_df.loc[0,'sale_duration'] = 16.0
# rf.predict(query_df)
# rf.predict_proba(query_df)
p = rf.predict_proba(query_df[colls])[0,1]
l = float(p>0.5)
p

0.0

In [134]:
X.dtypes[X.dtypes == 'int64']

approx_payout_date    int64
body_length           int64
channels              int64
event_created         int64
event_end             int64
event_start           int64
fb_published          int64
has_analytics         int64
has_logo              int64
name_length           int64
num_order             int64
num_payouts           int64
object_id             int64
sale_duration2        int64
show_map              int64
user_age              int64
user_created          int64
user_type             int64
dtype: object

In [84]:
X['delivery_method'] = X.delivery_method.fillna(0.0)

In [48]:
def encode_onehot(df, cols):
    """
    One-hot encoding is applied to columns specified in a pandas DataFrame.
    
    Modified from: https://gist.github.com/kljensen/5452382
    
    Details:
    
    http://en.wikipedia.org/wiki/One-hot
    http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
    
    @param df pandas DataFrame
    @param cols a list of columns to encode
    @return a DataFrame with one-hot encoding
    """
    vec = DictVectorizer()
    
#     vec_data = pd.DataFrame(vec.fit_transform(df[cols].to_dict(outtype='records')).toarray())
    vec_data = pd.DataFrame(vec.fit_transform(df[cols].to_dict()).toarray())
    vec_data.columns = vec.get_feature_names()
    vec_data.index = df.index
    
    df = df.drop(cols, axis=1)
    df = df.join(vec_data)
    return df